In [ ]:
pip install transformers[torch]==4.26.1 accelerate==0.21.0 adapter-transformers huggingface_hub==0.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 49.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (5

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers.adapters import AdapterConfig


/usr/local/lib/python3.10/dist-packages/transformers/adapters/__init__.py:27: FutureWarning: The `adapter-transformers` package is deprecated and replaced by the `adapters` package. See https://docs.adapterhub.ml/transitioning.html.
  warnings.warn(


In [ ]:
import pandas as pd

# URL de ejemplo
url = 'https://raw.githubusercontent.com/skyteck8/BigDataMachineLearning/main/parrafos16.csv'

# Cargar el DataFrame desde la URL
df = pd.read_csv(url, encoding='utf-16')

# Mostrar las primeras filas del DataFrame
print(df.head())

                                             parrafo
0          Meditaciones   Por   Marco Aurelio    ...
1   De mi abuelo Vero: el buen carácter y la sere...
2   De la reputación y memoria legadas por mi pro...
3   De mi madre: el respeto a los dioses, la gene...
4   De mi bisabuelo: el no haber frecuentado las ...


In [ ]:
df["parrafo"]

0               Meditaciones   Por   Marco Aurelio    ...
1        De mi abuelo Vero: el buen carácter y la sere...
2        De la reputación y memoria legadas por mi pro...
3        De mi madre: el respeto a los dioses, la gene...
4        De mi bisabuelo: el no haber frecuentado las ...
                              ...                        
1583                                      «Bien has dicho
1584     Pero en la vida los tres actos son un drama c...
1585    » Porque fija el término aquel que un día fue ...
1586                 Tú eres irresponsable en ambos casos
1587     Vete, pues, con ánimo propicio, porque el que...
Name: parrafo, Length: 1588, dtype: object

In [ ]:
import pdb
# Cargar el tokenizer y el modelo preentrenado
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Agregar token de padding al tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Cargar la configuración "pfeiffer" para los adaptadores
config = AdapterConfig.load("pfeiffer")

# Agregar adaptadores al modelo usando la configuración "pfeiffer"
model.add_adapter("stoic_adapter", config=config)

# Entrenar solo los adaptadores
model.train_adapter("stoic_adapter")

# Función de tokenización
def tokenize_function(examples):
    # pdb.set_trace()
    tokenized_inputs = tokenizer(examples, padding="max_length", truncation=True, max_length=512)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy()
    return tokenized_inputs

# Tokenizar el dataset
tokenized_datasets = df["parrafo"].apply(tokenize_function)

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir='./logs',
)

# Crear el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

# Entrenar el modelo
trainer.train()

# Guardar el modelo ajustado
model.save_adapter("./stoic_gpt2_adapter", "stoic_adapter")
tokenizer.save_pretrained('./stoic_gpt2_adapter')

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1588
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 2382
  Number of trainable parameters = 894528


Epoch,Training Loss,Validation Loss
1,0.774500,0.453834
2,0.470600,0.442189
3,0.456100,0.440462


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1588
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Configuration saved in ./results/checkpoint-1000/generation_config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Configuration saved in ./results/checkpoint-1500/generation_config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1588
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/con

('./stoic_gpt2_adapter/tokenizer_config.json',
 './stoic_gpt2_adapter/special_tokens_map.json',
 './stoic_gpt2_adapter/vocab.json',
 './stoic_gpt2_adapter/merges.txt',
 './stoic_gpt2_adapter/added_tokens.json')

In [ ]:
from google.colab import drive
import os
import shutil

# Montar Google Drive
drive.mount('/content/drive')

# Verificar la existencia de la carpeta
folder_path = '/content/stoic_gpt2_adapter'
if os.path.exists(folder_path):
    print("La carpeta existe.")
else:
    print("La carpeta no existe.")
    # Terminar el proceso si la carpeta no existe
    exit()

# Especificar la ruta en Google Drive
destination_path = '/content/drive/My Drive/stoic_gpt2_adapter'

# Copiar la carpeta a Google Drive
shutil.copytree(folder_path, destination_path)

# Verificar que la carpeta se ha copiado correctamente
if os.path.exists(destination_path):
    print(f'Carpeta guardada en: {destination_path}')
else:
    print("Error al guardar la carpeta.")

Mounted at /content/drive
La carpeta existe.
Carpeta guardada en: /content/drive/My Drive/stoic_gpt2_adapter


In [ ]:
from transformers import pipeline
# Probar el modelo directamente con inputs
# Cargar el modelo y el tokenizer ajustados
model = GPT2LMHeadModel.from_pretrained('./stoic_gpt2_adapter')
tokenizer = GPT2Tokenizer.from_pretrained('./stoic_gpt2_adapter')

# Crear un pipeline de generación de texto
faq_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Función para interactuar con el chatbot
def chat_with_bot(question):
    prompt = f"Question: {question} Answer:"
    response = faq_pipeline(prompt, max_length=150, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, "").strip()
    return answer

# Ejemplo de uso
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            break
        bot_response = chat_with_bot(user_input)
        print(f"Bot: {bot_response}")

OSError: ./stoic_gpt2_adapter does not appear to have a file named config.json. Checkout 'https://huggingface.co/./stoic_gpt2_adapter/None' for available files.

In [ ]:
!ls /content/stoic_gpt2_adapter

adapter_config.json  merges.txt		  pytorch_model_head.bin   tokenizer_config.json
head_config.json     pytorch_adapter.bin  special_tokens_map.json  vocab.json


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import AdapterConfig

model_path = '/content/stoic_gpt2_adapter'

# Verificar si la ruta existe
import os
if not os.path.exists(model_path):
    raise OSError(f"La ruta especificada no existe: {model_path}")

# Cargar el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Cargar la configuración del adaptador
adapter_config = AdapterConfig.load(model_path + '/adapter_config.json')

# Cargar el modelo base GPT-2
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Cargar los pesos del adaptador
model.load_adapter(model_path + '/pytorch_adapter.bin', config=adapter_config)

# Cargar los pesos de la cabeza
model.load_head(model_path + '/pytorch_model_head.bin', config=model_path + '/head_config.json')

# Probar el modelo con un input de ejemplo
input_text = "Your example input text here"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids)
print(tokenizer.decode(output[0], skip_special_tokens=True))

loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_

OSError: Unable to load adapter /content/stoic_gpt2_adapter/pytorch_adapter.bin from any source. Please check the name of the adapter or the source.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdapterConfig, setup_adapter_config, AdapterType
import os

model_path = '/content/stoic_gpt2_adapter'

# Verificar si la ruta existe
if not os.path.exists(model_path):
    raise OSError(f"La ruta especificada no existe: {model_path}")

# Cargar el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Cargar la configuración del adaptador
adapter_config_path = os.path.join(model_path, 'adapter_config.json')
adapter_config = AdapterConfig.load(adapter_config_path)

# Cargar el modelo base GPT-2
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Añadir y cargar el adaptador
adapter_name = 'stoic_adapter'
model.add_adapter(adapter_name, config=adapter_config)
adapter_weights_path = os.path.join(model_path, 'pytorch_adapter.bin')
model.load_adapter(adapter_weights_path, load_as=adapter_name)

# Cargar los pesos de la cabeza (si es necesario)
head_weights_path = os.path.join(model_path, 'pytorch_model_head.bin')
model.load_head(head_weights_path)

# Activar el adaptador
model.set_active_adapters(adapter_name)

# Probar el modelo con un input de ejemplo
input_text = "Your example input text here"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
output = model.generate(input_ids)
print(tokenizer.decode(output[0], skip_special_tokens=True))

ImportError: cannot import name 'setup_adapter_config' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

# Cargar el modelo GPT-2 base
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Cargar el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Cargar el adapter
adapter_path = './stoic_gpt2_adapter'
model.load_adapter(adapter_path)

# Activar el adapter
model.set_active_adapters('default')

# Crear un pipeline de generación de texto
faq_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Función para interactuar con el chatbot
def chat_with_bot(question):
    prompt = f"Question: {question} Answer:"
    response = faq_pipeline(prompt, max_length=150, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, "").strip()
    return answer

# Ejemplo de uso
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            break
        bot_response = chat_with_bot(user_input)
        print(f"Bot: {bot_response}")


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "use_cach

ValueError: No adapter with name 'default' found. Please make sure that all specified adapters are correctly loaded.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline

# Cargar el modelo GPT-2 base
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Cargar el tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Cargar el adapter
adapter_path = './stoic_gpt2_adapter'
model.load_adapter(adapter_path, config="pfeiffer", set_active=True)

# Listar los adapters disponibles
print("Adapters disponibles:", model.config.adapters.adapters)

# Activar el adapter (verifica el nombre correcto)
model.set_active_adapters('stoic_adapter')  # Reemplaza 'stoic_adapter' con el nombre correcto si es diferente

# Crear un pipeline de generación de texto
faq_pipeline = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Función para interactuar con el chatbot
def chat_with_bot(question):
    prompt = f"Question: {question} Answer:"
    response = faq_pipeline(prompt, max_length=150, num_return_sequences=1)
    answer = response[0]['generated_text'].replace(prompt, "").strip()
    return answer

# Ejemplo de uso
if __name__ == "__main__":
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "q"]:
            break
        bot_response = chat_with_bot(user_input)
        print(f"Bot: {bot_response}")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/607a30d783dfa663caf39e06633721c8d4cfcd7e/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.26.1",
  "use_cach

Adapters disponibles: {'stoic_adapter': '9076f36a74755ac4'}
You: hola


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1190: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: y en la creo de los más en las creos sándico
You: que opinas de la vida


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: we're fine here.
You: did u read marco aurelio?¿


Generate config GenerationConfig {
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Bot: no y seguido seguia de los que todo ciertúdas a de las creiembre, el triste en las creembras con algunos del mundo


KeyboardInterrupt: Interrupted by user